# Data Cleaning

In [77]:
import pandas as pd
import numpy as np
import pandas_profiling
import re
import Levenshtein as lev
import random

from datetime import datetime as dt
from functools import reduce

import warnings
warnings.filterwarnings("ignore")

In [53]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [78]:
data = pd.read_csv("gun-violence-data_01-2013_03-2018.csv")

# Index incidents using Incident ID
data.columns = ['Incident ID', 'Date', 'State', 'City/County', 'Address', 'Nr Killed', 'Nr Injured',
           'Incident URL', 'Source URL', 'Incident URL Fields Missing',
           'Congressional District', 'Gun Stolen', 'Gun Type',
           'Incident Characteristics', 'Latitude', 'Location Description',
           'Longitude', 'Nr Guns Involved', 'Notes', 'Participant Age',
           'Participant Age Group', 'Participant Gender', 'Participant Name',
           'Participant Relationship', 'Participant Status', 'Participant Type',
           'Sources', 'State House District', 'State Senate District']
data.set_index("Incident ID", inplace=True)

In [289]:
pandas_profiling.ProfileReport(data)

In [79]:
# Add the Las Vegas shooting incident from 2017
lv_incident = pd.read_csv("las-vegas-1-10-2017.csv")
lv_incident.set_index("Incident ID", inplace=True)
data = pd.concat([data, lv_incident])

In [80]:
# Delete columns that are irrelevant to the analysis
data = data.drop(["Incident URL Fields Missing", "Latitude", "Longitude", "Congressional District", 
                  "State House District", "State Senate District", "Incident URL", "Source URL", 
                  "Sources"], axis=1)

In [81]:
# Convert dates to datetime objects
data["Date"] = pd.to_datetime(data["Date"])

# Reorder newly created dataframe
data = data.sort_values(by=["Date"])

In [82]:
# Rearrange the columns for readibility
data = data[['Date', 'State', 'City/County', 'Address', 'Location Description', 'Nr Killed', 'Nr Injured', 
             'Incident Characteristics', 'Nr Guns Involved', 'Gun Stolen', 'Gun Type', 'Notes', 
             'Participant Name', 'Participant Age', 'Participant Age Group', 'Participant Gender', 
             'Participant Relationship', 'Participant Status', 'Participant Type']]

In [83]:
# Function to split a dictionary string into a Python dictinary
def split_into_str_dict(string):
    if pd.isnull(string):
        return np.nan
    str_dict = dict()
    # Split the string by '|' or '||', depending on the format
    list = '#$%'.join('#$%'.join(string.split('||')).split('|')).split('#$%')
    for index in list:
        # If indices and values are split by '::'
        i, sep, v = index.partition('::')
        if sep and v:
            str_dict[int(i)] = v
        # If indices and values are split by ':'
        else:
            i, _, v = index.partition(':')
            str_dict[int(i)] = v
    return str_dict

In [84]:
# Function to split a list string into a Python list
def split_into_list(string):
    if pd.isnull(string):
        return np.nan
    # Split the string by '|' or '||', depending on the format
    return '#$%'.join('#$%'.join(string.split('||')).split('|')).split('#$%')

In [85]:
# Function to convert a dictionary's values to int
def convert_dict_to_int(str_dict):
    if pd.isnull(str_dict):
        return np.nan
    for key in str_dict:
        str_dict[key] = int(str_dict[key])
    return str_dict

In [86]:
# Apply the previous 3 functions to the relevant columns in the dataset
data[['Gun Stolen', 'Gun Type', 'Participant Name', 'Participant Age', 'Participant Age Group', 
      'Participant Gender', 'Participant Relationship', 'Participant Status', 
      'Participant Type']] = data[['Gun Stolen', 'Gun Type', 'Participant Name', 
                                   'Participant Age', 'Participant Age Group', 
                                   'Participant Gender', 'Participant Relationship', 
                                   'Participant Status', 'Participant Type']].applymap(split_into_str_dict)
data[['Participant Age']] = data[['Participant Age']].applymap(convert_dict_to_int)
data[['Incident Characteristics']] = data[['Incident Characteristics']].applymap(split_into_list)

In [87]:
# Lookup dictionary of common address abbreviations found in the dataset
lookup_dict = {"Pkwy": "Parkway", "Blvd": " Boulevard", "Rd": "Road", "Dr ": "Drive ",
               "Dr$": "Drive", "Ave ": "Avenue ", "Ave$": "Avenue", "avenues": "Avenues",
               "Block": "block", "Expwy": "Expressway", "Pl ": "Place ", "Pl$": "Place", 
               "Ln": "Lane", "Northwest": "NW", "Southwest": "SW", "Northeast": "NE",
               "Southeast": "SE", "North-west": "NW", "South-west": "SW", "North-east": "NE",
               "South-east": "SE", "NorthWest": "NW", "SouthWest": "SW", "NorthEast": "NE",
               "SouthEast": "SE", "North-West": "NW", "South-West": "SW", "North-East": "NE",
               "South-East": "SE", "North ": "N ", "South ": "S ", "East ": "E ", "West ": "W ",
               "North$": "N", "South$": "S", "East$": "E", "West$": "W", "Av ": "Avenue ", 
               "Av$": "Avenue", "St$": "Street"}

In [88]:
# Function to format white spaces neatly in an address string
def clean_whitespaces(address):
    # Remove double spaces
    new_address = re.sub(' +', ' ', address)
    # Remove spaces before comma
    new_address = re.sub(' ,', ',', new_address)
    # Revove spaces at the beginning and end of an address
    new_address = re.sub(r'^\s+', '', new_address, flags=re.UNICODE)
    new_address = re.sub(r"\s+$", "", new_address, flags=re.UNICODE)
    return new_address

In [89]:
# Function to remove special characters such as dashes (-), commas (,), or slashes (/) from an address string
def clean_special_characters(address):
    # Remove periods
    new_address = re.sub("\.", "", address)
    # Remove commas, dashes, slashes
    new_address = re.sub(",|\-|\/", " ", new_address)
    return new_address

In [90]:
# Function to format strings and apply lookup_dict rules
# TODO write a cython version of this function
def clean_address_string(address):
    if pd.isnull(address):
        return np.nan
    else:
        new_address = clean_special_characters(address)
        # Apply lookup_dict rules
        new_address = reduce(lambda x, y: re.sub(y, lookup_dict[y], x), lookup_dict, new_address)
        new_address = clean_whitespaces(new_address)
        return new_address

In [80]:
# Function to remove house/street numbers from address
def clean_unique_address(address):
    # Remove orders
    address = re.sub('([0-9]*th|[0-9]*1st|[0-9]*2nd|[0-9]*3rd)', ' ', address)
    # Remove digits
    address = re.sub('[0-9]*', '', address)
    # Remove cardinal points
    address = re.sub('N |E |S |W |NE|NW|SE|SW', ' ', address)
    # Clean whitespaces
    address = clean_whitespaces(address)
    return address

In [81]:
# Compare addresses only after removing numbers to ensure that the dissimilarity is not caused by houses/streets of different numbers
lev_addresses = dict()
for item in list(set(data[pd.notnull(data['Address'])]["Address"].apply(clean_address_string))):
    lev_addresses[index] = clean_unique_address(item)
    index += 1

In [83]:
# Sample 1% of the addresses
lev_addresses = dict(random.sample(lev_addresses.items(), 1884))

In [443]:
# Find combinations of addresses that have between 0.95 and 0.99 similarity, disregarding numbers of streets and cardinals
for sampled_address in sample_lev_addresses.keys():
    for address in unique_addresses.keys():
        similarity = lev.ratio(lev_addresses[sampled_address], lev_addresses[address])
        if similarity*100 in range(95, 100):
            print("There is a similarity of " + str(similarity) + " between the following strings:")
            print(" 1. " + unique_addresses[sampled_address])
            print(" 2. " + unique_addresses[address])

There is a similarity of 0.95 between the following strings:
 1. 1800 block of Amber Drive
 2. 3000 block of Ember Drive
There is a similarity of 0.96 between the following strings:
 1. N Clark Street
 2. 1812 W Clarke Street
There is a similarity of 0.96 between the following strings:
 1. N Clark Street
 2. 1820 W Clarke Street
There is a similarity of 0.96 between the following strings:
 1. N Clark Street
 2. Clarke Street
There is a similarity of 0.96 between the following strings:
 1. N Clark Street
 2. 1617 W Clarke Street
There is a similarity of 0.96 between the following strings:
 1. N Clark Street
 2. 3900 Clarke Street
There is a similarity of 0.95 between the following strings:
 1. 1000 block of Eagle Drive
 2. 1950 block of Cagle Drive
There is a similarity of 0.96 between the following strings:
 1. 202 W Baker Street
 2. 185 Barker Street
There is a similarity of 0.96 between the following strings:
 1. 202 W Baker Street
 2. 75 Barker Street
There is a similarity of 0.96 b

There is a similarity of 0.96 between the following strings:
 1. 304 Waters Avenue
 2. S Water Avenue
There is a similarity of 0.96 between the following strings:
 1. 304 Waters Avenue
 2. Water Avenue
There is a similarity of 0.96 between the following strings:
 1. 304 Waters Avenue
 2. NW Water Avenue
There is a similarity of 0.96 between the following strings:
 1. 6400 block of S Claremont Avenue
 2. 2900 block of Clairmont Avenue
There is a similarity of 0.95 between the following strings:
 1. 2200 block of Poland Road
 2. 8300 block of Noland Road
There is a similarity of 0.95 between the following strings:
 1. 2200 block of Poland Road
 2. 9700 block of Noland Road
There is a similarity of 0.95 between the following strings:
 1. 2200 block of Poland Road
 2. 1100 block of N Noland Road
There is a similarity of 0.95 between the following strings:
 1. 1100 block of Oster Drive
 2. 100 block of Otero Drive
There is a similarity of 0.95 between the following strings:
 1. 1100 block o

There is a similarity of 0.96 between the following strings:
 1. 800 block of Martin Luther King Boulevard
 2. 300 block of Martin Luther King Jr Boulevard
There is a similarity of 0.96 between the following strings:
 1. 800 block of Martin Luther King Boulevard
 2. 2600 block of Martin Luther King Jr Boulevard SW
There is a similarity of 0.96 between the following strings:
 1. 800 block of Martin Luther King Boulevard
 2. 100 block of Martin Luther King Jr Boulevard
There is a similarity of 0.96 between the following strings:
 1. 800 block of Martin Luther King Boulevard
 2. 300 block of N Martin Luther King Jr Boulevard
There is a similarity of 0.96 between the following strings:
 1. 800 block of Martin Luther King Boulevard
 2. 800 block of S Martin Luther King Jr Boulevard
There is a similarity of 0.96 between the following strings:
 1. 800 block of Martin Luther King Boulevard
 2. 600 block of Martin Luther King Jr Boulevard
There is a similarity of 0.96 between the following stri

There is a similarity of 0.95 between the following strings:
 1. 1700 block of Eden Avenue
 2. 10300 block of Edes Avenue
There is a similarity of 0.95 between the following strings:
 1. 1700 block of Eden Avenue
 2. 8400 block of Edes Avenue
There is a similarity of 0.95 between the following strings:
 1. 1700 block of Eden Avenue
 2. 2300 block of Edna Avenue
There is a similarity of 0.95 between the following strings:
 1. 1700 block of Eden Avenue
 2. 5900 block of Edna Avenue
There is a similarity of 0.95 between the following strings:
 1. 1700 block of Eden Avenue
 2. 9800 block of Edes Avenue
There is a similarity of 0.95 between the following strings:
 1. 1700 block of Eden Avenue
 2. 10400 block of Edes Avenue
There is a similarity of 0.96 between the following strings:
 1. 615 N Hawley Street
 2. Haley Street
There is a similarity of 0.96 between the following strings:
 1. 512 S Kings Highway
 2. 5937 King Highway
There is a similarity of 0.96 between the following strings:
 1

There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 3100 block of High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 200 block of High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 1700 block of High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 10000 block of High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 400 block of High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 3400 block of High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 1100 block of E High Street
There is a similarity of 0.95 between the following strings:
 1. 2600 block of Eighth Street NE
 2. 900 block of High

There is a similarity of 0.96 between the following strings:
 1. 6422 Storer Avenue
 2. 92 Store Avenue
There is a similarity of 0.95 between the following strings:
 1. 1800 block of Pryor Street
 2. 4200 block of Pro Street
There is a similarity of 0.95 between the following strings:
 1. 1500 block of SW Polk Street
 2. 300 block of Folk Street
There is a similarity of 0.96 between the following strings:
 1. Cricket Lane
 2. 20 Crickett Lane
There is a similarity of 0.96 between the following strings:
 1. Elliot Street
 2. 36 Eliot Street
There is a similarity of 0.95 between the following strings:
 1. 500 block of Iowa Avenue
 2. 3200 block of Iola Avenue
There is a similarity of 0.96 between the following strings:
 1. 1214 Baker Avenue
 2. 4969 N Backer Avenue
There is a similarity of 0.96 between the following strings:
 1. 1214 Baker Avenue
 2. N Backer Avenue
There is a similarity of 0.96 between the following strings:
 1. 1214 Baker Avenue
 2. 273 Barker Avenue
There is a similar

In [91]:
data[['Address']] = data[['Address']].applymap(clean_address_string)

In [92]:
data[["Suspect Name", "Suspect Age", "Suspect Age Group", "Suspect Gender", "Suspect Relationship", "Suspect Status", "Nr Suspects"]] = pd.DataFrame([[np.nan for _ in range(7)] for _ in range(data.shape[0])], index=data.index)

In [93]:
def populate_suspect_columns(key, row):
    for col in range(12, 18):
        if pd.notnull(row[col]):
            if key in row[col].keys():
                if pd.isnull(row[col+7]):
                    row[col+7] = dict()
                row[col+7][key] = row[col][key]
                del row[col][key]

In [94]:
def find_suspect_key(row):
    nr_suspects = 0
    if pd.notnull(row[18]):
        for k, v in row[18].items():
            if v == "Subject-Suspect":
                nr_suspects += 1
                populate_suspect_columns(k, row)
    row[25] = nr_suspects
    return row
            
def append_suspect_columns(df, index, columns):
    return pd.DataFrame([find_suspect_key(df[i]) for i in range(df.shape[0])], index=index, columns=columns)

In [95]:
data_list = data.values
data = append_suspect_columns(data_list, data.index, data.columns)

In [96]:
def replace_dict(dictionary):
    if pd.notnull(dictionary):
        return len(dictionary)
    else:
        return np.nan
    
data["Participant Type"] = data["Participant Type"].apply(lambda x: replace_dict(x))
data.rename(columns = {"Participant Type": "Nr Victims"}, inplace = True)

In [97]:
for suffix in ["Name", "Age", "Age Group", "Gender", "Relationship", "Status"]:
    data.rename(columns = {"Participant " + suffix: "Victim " + suffix}, inplace = True)

In [99]:
for year in range(2013, 2019):
    print("Total number of incidents recorded in " + str(year) + ": \t"+ str(len(data[data["Date"].dt.year == year])))
print("-------------------------------------------------------")
print("Total number of incidents recorded: \t\t" + str(len(data)))
print("-------------------------------------------------------")
print("Average number of incidents recorded (2014-17): " + str(round(len(data[data["Date"].dt.year.between(2014, 2018)]) / 4)))

Total number of incidents recorded in 2013: 	278
Total number of incidents recorded in 2014: 	51854
Total number of incidents recorded in 2015: 	53579
Total number of incidents recorded in 2016: 	58763
Total number of incidents recorded in 2017: 	61402
Total number of incidents recorded in 2018: 	13802
-------------------------------------------------------
Total number of incidents recorded: 		239678
-------------------------------------------------------
Average number of incidents recorded (2014-17): 59850


In [100]:
def get_yearly_casualties(row):
    return max(row[5] + row[6] - row[24], 0)

In [101]:
def yearly_casualties(data, year):
    return [get_yearly_casualties(data[i]) for i in range(data.shape[0]) if data[i][0].year == 2013]

In [102]:
casualties_2013 = yearly_casualties(data.values, 2013)

TypeError: unsupported operand type(s) for -: 'int' and 'dict'

In [ ]:
print("Total number of mass shootings in 2013: " + str(len([nr_casualties for nr_casualties in casualties_2013 if nr_casualties >= 4])))
print("Total number of incidents in 2013: \t" + str(len(casualties_2013)))

In [ ]:
# Create a mass shooting dataset for further analysis
mass_shootings = data[(data["Nr Killed"] + data["Nr Injured"] - data["Nr Suspects"]) >= 4]

# Remove 2013 incidents from the dataset
data = data[data["Date"].dt.year != 2013]